In [1]:
import sys
import os

# Add the path to the folder containing ClusterAndPredict
module_path = os.path.abspath(os.path.join('..', '/workspaces/FactChecker-V3'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from ClusterAndPredict.ClusterAndPredict import ClusterAndPredict
from Testing.DataLoader import DataLoader

2024-09-17 02:26:55.105592: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-17 02:26:55.109884: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-17 02:26:55.124406: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-17 02:26:55.147884: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-17 02:26:55.155032: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-17 02:26:55.171739: I tensorflow/core/platform/cpu_feature_gu

In [3]:
percentage = 0.75
data_loader = DataLoader(percentage, True, 23)
train_df, test_df = data_loader.create_train_test_df(True, True, True)

Numerical Rating
1    2980
3     620
Name: count, dtype: int64
Numerical Rating
1    778
3    131
Name: count, dtype: int64


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [4]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [4]:
from datasets import Dataset

train_df['label'] = train_df['Numerical Rating'].apply(lambda x: 0 if x == 1 else 1)
test_df['label'] = test_df['Numerical Rating'].apply(lambda x: 0 if x == 1 else 1)

# Rename the Text column to text
train_df = train_df.rename(columns={'Text': 'text'})
test_df = test_df.rename(columns={'Text': 'text'})

# Drop all columsn except text and label
train_df = train_df[['text', 'label']]
test_df = test_df[['text', 'label']]

train_df = Dataset.from_pandas(train_df)
test_df = Dataset.from_pandas(test_df)

# Use tokenizer to encode the train and test df and add the label
train_df = train_df.map(tokenize_function, batched=True)
test_df = test_df.map(tokenize_function, batched=True)

NameError: name 'tokenize_function' is not defined

In [5]:
train_df.to_csv('train.csv')
test_df.to_csv('test.csv')

Creating CSV from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

168266

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./resultsSOTA",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_df,
    eval_dataset=test_df,
    tokenizer=tokenizer,
)

trainer.train()

In [ ]:
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")